In [ ]:
"""
Purpose: To develop the methods for doing spine detection

"""

In [7]:
from os import sys
sys.path.append("../../meshAfterParty/")
from importlib import reload

In [8]:
import os
from pathlib import Path
os.getcwd()

'/notebooks/Platinum_Spine_vp2'

In [9]:
compressed_neuron_path = Path("../test_neurons/test_objects/12345_2_soma_practice_decompress")

import neuron_utils as nru
nru = reload(nru)
import neuron
neuron=reload(neuron)

import system_utils as su

with su.suppress_stdout_stderr():
    recovered_neuron = nru.decompress_neuron(filepath=compressed_neuron_path,
                      original_mesh=compressed_neuron_path)

recovered_neuron

# Previous Work

In [39]:
"""
Pseudocode: 
1) make sure the cgal temp folder exists
2) run the segmentation command
3) Read int csv 
4) Visualize the results using the graph function

"""
import cgal_Segmentation_Module as csm
from pathlib import Path
import trimesh_utils as tu
import numpy as np
import numpy_utils as nu
import skeleton_utils as sk
import copy

def split_mesh_into_spines_shaft(current_mesh,
                           segment_name="",
                           clusters=2,
                          smoothness=0.03,
                          cgal_folder = Path("./cgal_temp"),
                          delete_temp_file=True,
                          shaft_threshold = 300):

    if not cgal_folder.exists():
        cgal_folder.mkdir(parents=True,exist_ok=False)

    file_to_write = cgal_folder / Path(f"segment_{segment_name}.off")

    written_file_location = tu.write_neuron_off(current_mesh,file_to_write)

    if written_file_location[-4:] == ".off":
        cgal_mesh_file = written_file_location[:-4]
    else:
        cgal_mesh_file = written_file_location

    print(f"Going to run cgal segmentation with:"
         f"\nFile: {cgal_mesh_file} \nclusters:{clusters} \nsmoothness:{smoothness}")

    csm.cgal_segmentation(cgal_mesh_file,clusters,smoothness)

    #read in the csv file
    cgal_output_file = Path(cgal_mesh_file + "-cgal_" + str(np.round(clusters,2)) + "_" + "{:.2f}".format(smoothness) + ".csv" )

    cgal_data = np.genfromtxt(str(cgal_output_file.absolute()), delimiter='\n')

    #get a look at how many groups and what distribution:
    from collections import Counter
    print(f"Counter of data = {Counter(cgal_data)}")

    split_meshes,split_meshes_idx = tu.split_mesh_into_face_groups(current_mesh,cgal_data,return_idx=True,
                                   check_connect_comp = False)
    
    
    if len(split_meshes.keys()) <= 1:
        print("There was only one mesh found from the spine process and mesh split, returning empty array")
        return dict(),dict()
        
    
    # How to identify just one shaft
    shaft_index = -1
    shaft_total = -1
    for k,v in split_meshes.items():
        curr_length = len(v.faces)
        if  curr_length > shaft_total:
            shaft_index = k
            shaft_total = curr_length
    
    shaft_mesh = split_meshes.pop(shaft_index)
    shaft_mesh_idx = split_meshes_idx.pop(shaft_index)
    
    print(f"shaft_index = {shaft_index}")
    
    #Applying a length threshold to get all other possible shaft meshes
    
 

    if delete_temp_file:
        file_to_write.unlink()
        
    return split_meshes,split_meshes_idx,shaft_mesh,shaft_mesh_idx

current_neuron = recovered_neuron
branch_idx = 0
limb_idx = 0
current_mesh = recovered_neuron.concept_network.nodes[nru.limb_label(limb_idx)]["data"].concept_network.nodes[branch_idx]["data"].mesh
spine_data_returned= split_mesh_into_spines_shaft(current_mesh,
                           segment_name=f"{limb_idx}_{branch_idx}",
                           clusters=2,
                          smoothness=0.03,
                          cgal_folder = Path("./cgal_temp"),
                          delete_temp_file=True)

if len(spine_data_returned[0].keys()) < 0:
    print("No spines were found")
else:
    spine_meshes,spine_meshes_idx,shaft_mesh,shaft_mesh_idx = spine_data_returned
     
    sk.graph_skeleton_and_mesh(
                               #main_mesh_verts = shaft_mesh.vertices,
                               #main_mesh_faces = shaft_mesh.faces,
                               #main_mesh_color="black",
                              other_meshes=[k for k in spine_meshes.values()],
                              other_meshes_colors="random",
                               mesh_alpha=1)
    sk.graph_skeleton_and_mesh(
                           main_mesh_verts = shaft_mesh.vertices,
                           main_mesh_faces = shaft_mesh.faces,
                           main_mesh_color="black"
    )
    
    
    

Going to run cgal segmentation with:
File: /notebooks/Platinum_Spine_vp2/cgal_temp/segment_0_0 
clusters:2 
smoothness:0.03
Counter of data = Counter({28.0: 2181, 26.0: 1506, 29.0: 561, 23.0: 326, 24.0: 143, 6.0: 134, 25.0: 109, 27.0: 89, 21.0: 80, 12.0: 76, 0.0: 74, 1.0: 62, 9.0: 61, 16.0: 54, 22.0: 51, 11.0: 48, 18.0: 45, 15.0: 35, 4.0: 34, 2.0: 33, 14.0: 25, 8.0: 22, 19.0: 21, 13.0: 21, 3.0: 20, 20.0: 19, 10.0: 18, 17.0: 13, 7.0: 6, 5.0: 6})



shaft_index = 28.0


# Determining right thresholds

In [46]:
total_keys = list(spine_meshes.keys())

In [49]:
curr_key_index = 3
k = total_keys[curr_key_index]
k = 29.0
v = spine_meshes[k]

print(f"Index {curr_key_index}, Spine {k}: face length = {len(v.faces)}")
sk.graph_skeleton_and_mesh(other_meshes = [current_mesh,v],
                          other_meshes_colors=["black","red"],
                          mesh_alpha=1)

Index 3, Spine 29.0: face length = 561


In [ ]:
spine_indexes = [0,1,2]
shaft_indexes = [ ]

In [ ]:
"""
Problem going to face:
1) The shaft gets split up into multiple pieces (just do a size threshold)
2) but could have a large merge? but then would be part of another branch 
3) 
"""

In [ ]:
"""
Pseudocode: 
1) make sure the cgal temp folder exists
2) run the segmentation command
3) Read int csv 
4) Visualize the results using the graph function

Notes on what worked well:
segment = 56: cluster=2,smoothness=0.03

"""
import cgal_Segmentation_Module as csm

clusters=2
smoothness = 0.03
check_index = 56

from pathlib import Path
cgal_folder = Path("./cgal_temp")
if not cgal_folder.exists():
    cgal_folder.mkdir(parents=True,exist_ok=False)


current_mesh = total_branch_meshes[check_index]

file_to_write = cgal_folder / Path(f"segment_{check_index}.off")

written_file_location = tu.write_neuron_off(current_mesh,file_to_write)

if written_file_location[-4:] == ".off":
    cgal_mesh_file = written_file_location[:-4]
else:
    cgal_mesh_file = written_file_location
    
print(f"Going to run cgal segmentation with:"
     f"\nFile: {cgal_mesh_file} \nclusters:{clusters} \nsmoothness:{smoothness}")
    
csm.cgal_segmentation(cgal_mesh_file,clusters,smoothness)

#read in the csv file
cgal_output_file = Path(cgal_mesh_file + "-cgal_" + str(np.round(clusters,2)) + "_" + "{:.2f}".format(smoothness) + ".csv" )

cgal_data = np.genfromtxt(str(cgal_output_file.absolute()), delimiter='\n')

#get a look at how many groups and what distribution:
from collections import Counter
print(f"Counter of data = {Counter(cgal_data)}")

split_meshes,split_meshes_idx = tu.split_mesh_into_face_groups(current_mesh,cgal_data,return_idx=True,
                               check_connect_comp = False)

split_meshes,split_meshes_idx
# plot the face mapping 
sk.graph_skeleton_and_mesh(other_meshes=[k for k in split_meshes.values()],
                          other_meshes_colors="random",
                           mesh_alpha=1)

In [ ]:
# Current Work 